In [4]:
import pandas as pd
import requests
import json
import csv
import os
import time

In [15]:
# API Params
query = '' # What are you searching
location = '' # Where are you searching
apikey = '' #Google API key


In [16]:
def save_results():
    if not os.path.isfile('Results.csv'): # If file exist - headers are not saved, file is appended
        total_page3.to_csv('Results.csv', mode='a', header=True, index=False)
    else:    # if file exists - headers are not inlucded and file is appended                              
        total_page3.to_csv('Results.csv', mode='a', header=False, index=False) #Index removed as it's iterating again in each page

if location and apikey:
    url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query={},{}&key={}'.format(query, location, apikey)
    results = requests.get(url).json()
    df = pd.json_normalize(results['results'])
    token = results.get("next_page_token",None)    
    time.sleep(2)
    if token:
        url_page2 = 'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={}&key={}'.format(token, apikey)
        results_page2 = requests.get(url_page2).json()
        df_page2 = pd.json_normalize(results_page2['results'])
        total_page2 = df.append(df_page2)
        token_page2 = results_page2.get("next_page_token",None)
        time.sleep(2)
        if token_page2:
            url_page3 = 'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={}&key={}'.format(token_page2, apikey)
            results_page3 = requests.get(url_page3).json()
            df_page3 = pd.json_normalize(results_page3['results'])
            total_page3 = total_page2.append(df_page3)
            save_results()
    else:
        print('No next page token')
        save_results()
else:
    print('Apikey or location missing')